In [1]:
from clickzetta.zettapark.session import Session

In [2]:
import json
# 从配置文件中读取参数
with open('security/config-uat.json', 'r') as config_file:
    config = json.load(config_file)

# 创建会话
session = Session.builder.configs(config).create()

In [4]:
data = [('James','Smith','M',3000),('Anna','Rose','F',4100),
  ('Robert','Williams','NA',6200),(None,'Rob','F',6200)
  
]

columns = ["firstname","lastname","gender","salary"]
df = session.createDataFrame(data=data, schema = columns)
df.show()

------------------------------------------
|firstname  |lastname  |gender  |salary  |
------------------------------------------
|James      |Smith     |M       |3000    |
|Anna       |Rose      |F       |4100    |
|Robert     |Williams  |NA      |6200    |
|NULL       |Rob       |F       |6200    |
------------------------------------------



In [5]:
df2=df.withColumn("salary", df.salary*3)
df2.show()

------------------------------------------
|firstname  |lastname  |gender  |salary  |
------------------------------------------
|James      |Smith     |M       |9000    |
|Anna       |Rose      |F       |12300   |
|Robert     |Williams  |NA      |18600   |
|NULL       |Rob       |F       |18600   |
------------------------------------------



In [6]:
from clickzetta.zettapark.functions import when
df3 = df.withColumn("gender", when(df.gender == "M","Male") \
      .when(df.gender == "F","Female") \
      .otherwise(df.gender))
df3.show()

df4=df.withColumn("salary",df.salary.cast("String"))
df4.printSchema()

df.createOrReplaceTempView("PER")
df5=session.sql("select firstname,gender,salary*3 as salary from PER")
df5.show()

------------------------------------------
|firstname  |lastname  |salary  |gender  |
------------------------------------------
|James      |Smith     |3000    |Male    |
|Anna       |Rose      |4100    |Female  |
|Robert     |Williams  |6200    |NA      |
|NULL       |Rob       |6200    |Female  |
------------------------------------------

root
 |-- `firstname`: StringType() (nullable = True)
 |-- `lastname`: StringType() (nullable = False)
 |-- `gender`: StringType() (nullable = False)
 |-- `salary`: StringType() (nullable = False)
-------------------------------
|firstname  |gender  |salary  |
-------------------------------
|James      |M       |9000    |
|Anna       |F       |12300   |
|Robert     |NA      |18600   |
|NULL       |F       |18600   |
-------------------------------



In [7]:
session.close()